## **silver-companies-balance-sheet**
<p>This notebook extracts and stores raw balance sheet data for publicly listed companies from Yahoo Finance. It includes key financial figures such as assets, liabilities, and shareholder equity, forming the foundational layer for financial health and stability analysis.</p>

#### **Importing necessary libraries**

In [ ]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

#### **Variables**

In [39]:
file_name = "comp_bal_sheet_data"
layer_name = "bronze"

StatementMeta(, b8e8a1f6-990b-4cc2-919e-b313e10d46a3, 42, Finished, Available, Finished)

#### **Step 2: Read the Balance Sheet and Breakdown Categories Data**

In [40]:
bal_sheet_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

breakdown_categories = spark.read.csv("Files/master_files/breakdown_categories.csv", header=True)

StatementMeta(, b8e8a1f6-990b-4cc2-919e-b313e10d46a3, 43, Finished, Available, Finished)

#### **Step 3: Filters the Unnecessary Breakdown Categories from the Balance Sheet**

In [41]:
output_df = bal_sheet_df\
                    .join(broadcast(breakdown_categories), bal_sheet_df.Breakdown == breakdown_categories['Account Name'], 'inner')\
                    .select(bal_sheet_df.Breakdown, bal_sheet_df.Company, bal_sheet_df.Value.alias('amount'), bal_sheet_df.Date)

StatementMeta(, b8e8a1f6-990b-4cc2-919e-b313e10d46a3, 44, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [42]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        delta_table.alias("target").merge(
            output_df.alias("source"),
                    (col("target.Breakdown") == col("source.Breakdown")) &
                    (col("target.Company") == col("source.Company")) &
                    (col("target.Date") == col("source.Date"))
        ).whenMatchedUpdateAll(
        ).whenNotMatchedInsertAll().execute()

    else:
        output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, b8e8a1f6-990b-4cc2-919e-b313e10d46a3, 45, Finished, Available, Finished)